In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split
from torchtext.vocab import build_vocab_from_iterator

In [2]:
tkDF = pd.read_csv('../data/경상도.csv')

In [3]:
tkDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212906 entries, 0 to 212905
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사투리     212906 non-null  object
 1   표준어     212906 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


In [21]:
tkDF.head(20)

,variable,value
0,사투리,지금은 저는 비 오는 것보다는 눈을 더 적극적으로 원하는데요 대구에서는 요즘 눈이 ...
1,표준어,어제 저녁에는 간식을 너무 많이 먹어가지고 밥 생각이 별로 없어가지고 누룽지를 끓여...
2,표준어,다 큰 아들이 버스 정류장에서 넘어졌을 때 창피해 같아서 해야 할지 모르겠다
3,사투리,주차장 들어갈 때는 저 앞에 가서 우선 차를 세우고 어디 가시는지 말을 말려주면 됩니다.
4,표준어,내가 핸드폰을 사용할 수 있는 거는 다른 친구들 보다가 좀 기능을 많이 사용하는데 ...
5,표준어,건강을 위해 어 등산도 자주 다니고 많이 걷고 운동을 부지런히 하는 편인데 특히 어...
6,사투리,귀밝기 술을 한잔 마셨고요 귀가 그 밝아져서 말하는 걸 잘 듣 들으라고 하는 뜻으로...
7,사투리,옛날에 혼자 김장을 했는데 요즘은 힘들어가 동네 사람들하고 같이 합니다
8,사투리,비틀로 옷을 만들어가 입으셨다고 캤는디 옷 맹그는 과정을 알려주실랍니껴
9,사투리,길 건너다가 차에 째끔 저 박았는데 병원에 가서 진찰을 받아봐야 좋겠죠


In [5]:
tkDF = pd.melt(tkDF)

In [6]:
tkDF = tkDF.sample(frac=1).reset_index(drop=True)

In [86]:
# 중복값 제거
tkDF.drop_duplicates(inplace=True)

In [87]:
tkDF['variable'].values

array(['사투리', '표준어', '표준어', ..., '사투리', '표준어', '사투리'], dtype=object)

In [88]:
import string

tkDF['value'] = tkDF['value'].replace(r'[{}]'.format(string.punctuation), '', regex=True)

In [90]:
# 한글 불용어 제거 # 어근 제거는 안함 - 사투리와 표준어 구별 사라짐
with open('data/hangul_stopword.txt', 'r', encoding='utf-8') as f:
    stopword_h = [line.strip() for line in f.readlines()]

In [99]:
class dataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.DF = df
        self.label = [1 if value == '사투리' else 0 for value in self.DF['variable'].values]
        self.text = self.DF['value'].values

    def __len__(self): 
        return len(self.text)
    
    def __getitem__(self, idx):
        return self.label[idx],self.text[idx]

In [100]:
tkDS = dataset(tkDF)

In [101]:
train_length = int(0.8 * len(tkDS))
val_length = len(tkDS) - train_length
trainDS, valDS = random_split(tkDS, [train_length, val_length])

In [103]:
# 토큰 관련 특별 문자
unk = '<UNK>'
pad = '<PAD>'

In [115]:
from konlpy.tag import Mecab
mecab = Mecab()

In [120]:
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield mecab.morphs(text)

In [121]:
# !pip3 install mecab-ko-msvc mecab-ko-dic-msvc

In [122]:
VOCAB = build_vocab_from_iterator(yield_tokens(trainDS), specials=[unk, pad], special_first=True)

VOCAB.set_default_index(VOCAB[unk])

In [105]:
vocab = {}
for idx in tkDF.index:
    words = mecab.morphs(tkDF.loc[idx, 'value'])
    for word in words:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [106]:
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [107]:
vocabDF = pd.DataFrame(sorted_vocab, columns=['word', 'count'])
vocabDF.head(30)

,word,count
0,고,865676
1,는,653175
2,가,450936
3,이,445867
4,에,435452
5,하,419068
6,도,342788
7,은,282802
8,을,267262
9,있,221719


In [108]:
# 데이터셋용 단어사전 생성
VOCAB = {0:unk, 1:pad}

for idx in vocabDF.index:
    VOCAB[idx+2] = vocabDF.loc[idx, 'word']

In [110]:
# 텍스트 > 정수 인코딩
text_pipeline = lambda x: [VOCAB.get(word, 0) for word in mecab.morphs(x)]

In [111]:
# 배치 크기 만큼 데이터셋 반환 함수
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(_label)
        _text = text_pipeline(_text)
        text_list.append(_text)

    # 텐서화
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = torch.cat(text_list)

In [112]:
BATCH_SIZE = 64
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE, collate_fn=collate_batch)
testDL = DataLoader(valDS, batch_size=BATCH_SIZE, collate_fn=collate_batch)

In [113]:
NUM_CLASSES = 2
VOCAB_SIZE = len(VOCAB)

print(f'NUM_CLASSES: {NUM_CLASSES} VOCAB_SIZE: {VOCAB_SIZE}')

NUM_CLASSES: 2 VOCAB_SIZE: 49672


In [ ]:
class TextModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, sparse=False) # 밀집행렬 생성
        